In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.0.0-alpha0'

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

##### Build model to be saved and loaded

In [6]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    return model

model = create_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


###### tf.keras.callbacks.ModelCheckpoint  is a callback that you can use to save checkpoints DURING and at the END of training.

In [7]:
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

#create checkp callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1)
model = create_model()

model.fit(train_images, train_labels, epochs=10,
         validation_data =(test_images, test_labels),
         callbacks =[cp_callback])

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 864/1000 [========================>.....] - ETA: 0s - loss: 1.2416 - accuracy: 0.6389
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 774us/sample - loss: 1.1617 - accuracy: 0.6600 - val_loss: 0.7059 - val_accuracy: 0.7900
Epoch 2/10
 928/1000 [==========================>...] - ETA: 0s - loss: 0.4369 - accuracy: 0.8782
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 313us/sample - loss: 0.4285 - accuracy: 0.8800 - val_loss: 0.5511 - val_accuracy: 0.8310
Epoch 3/10
 928/1000 [==========================>...] - ETA: 0s - loss: 0.2783 - accuracy: 0.9321
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 318us/sample - loss: 0.2777 - accuracy: 0.9310 - val_loss: 0.4653 - val_accuracy: 0.8500
Epoch 4/10
 896/1000 [=========================>....] - ETA: 0s - loss: 0.2054 - accuracy: 0.9487
Epoch

In [10]:
ls {checkpoint_dir}

 Volume in drive C is OS
 Volume Serial Number is 9C33-2CCC

 Directory of C:\Users\Jessy\dev\machine-learning\TF 2.0 save and restore\training_1

04/17/2019  09:55 PM    <DIR>          .
04/17/2019  09:55 PM    <DIR>          ..
04/17/2019  09:55 PM                71 checkpoint
04/17/2019  09:55 PM         4,889,870 cp.ckpt.data-00000-of-00001
04/17/2019  09:55 PM             1,579 cp.ckpt.index
               3 File(s)      4,891,520 bytes
               2 Dir(s)  803,331,981,312 bytes free


##### Creating an untrained model and evaluating on test set.

In [11]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print('Untrained model, accuracy: {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 112us/sample - loss: 2.3114 - accuracy: 0.1140
Untrained model, accuracy: 11.40%


##### Now loading weights from checkpoint and evaluating again:

In [13]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print('Restored model, accuarcy: {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 37us/sample - loss: 0.4340 - accuracy: 0.8620
Restored model, accuarcy: 86.20%


In [15]:
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
#saves weight at every 5 epochs
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
         epochs=50, callbacks= [cp_callback],
         validation_data = (test_images, test_labels),
         verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [18]:
ls {checkpoint_dir}

 Volume in drive C is OS
 Volume Serial Number is 9C33-2CCC

 Directory of C:\Users\Jessy\dev\machine-learning\TF 2.0 save and restore\training_2

04/18/2019  12:02 PM    <DIR>          .
04/18/2019  12:02 PM    <DIR>          ..
04/18/2019  12:02 PM                81 checkpoint
04/18/2019  12:02 PM         1,631,620 cp-0000.ckpt.data-00000-of-00001
04/18/2019  12:02 PM               706 cp-0000.ckpt.index
04/18/2019  12:02 PM         4,889,915 cp-0005.ckpt.data-00000-of-00001
04/18/2019  12:02 PM             1,579 cp-0005.ckpt.index
04/18/2019  12:02 PM         4,889,915 cp-0010.ckpt.data-00000-of-00001
04/18/2019  12:02 PM             1,579 cp-0010.ckpt.index
04/18/2019  12:02 PM         4,889,915 cp-0015.ckpt.data-00000-of-00001
04/18/2019  12:02 PM             1,579 cp-0015.ckpt.index
04/18/2019  12:02 PM         4,889,915 cp-0020.ckpt.data-00000-of-00001
04/18/2019  12:02 PM             1,579 cp-0020.ckpt.index
04/18/2019  12:02 PM         4,889,915 cp-0025.ckpt.data-00000-of-0000

In [19]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

###### **** TF default format is to only save 5 most recent checkpoints****

In [20]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print('restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 93us/sample - loss: 0.4850 - accuracy: 0.8730
restored model, accuracy: 87.30%


##### Manually saving weights:

In [21]:
model.save_weights('./checkpoints/my_checkpoint')

model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 91us/sample - loss: 0.4850 - accuracy: 0.8730
Restored model, accuracy: 87.30%


##### Saving entire model as an HDF5 file:

You can save a fully functional model, then load in TF.js, then train and run them in web browsers, or convert to run on mobile with TF Lite

In [22]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

#save entire model to HDF5 file:
model.save('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 0s 214us/sample - loss: 1.1620 - accuracy: 0.6740
Epoch 2/5
1000/1000 [==============================] - 0s 109us/sample - loss: 0.4109 - accuracy: 0.8950
Epoch 3/5
1000/1000 [==============================] - 0s 107us/sample - loss: 0.2716 - accuracy: 0.9340
Epoch 4/5
1000/1000 [==============================] - 0s 105us/sample - loss: 0.2081 - accuracy: 0.9560
Epoch 5/5
1000/1000 [==============================] - 0s 109us/sample - loss: 0.1536 - accuracy: 0.9660


##### Recreating the model from that ^ file:

In [23]:
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [24]:
loss, acc = new_model.evaluate(test_images, test_labels)
print('Restored mode, accuracy: {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 94us/sample - loss: 0.4351 - accuracy: 0.8630
Restored mode, accuracy: 86.30%


This technique saves everything..  Weight values, model architecture, and optimizer config.  **CURRENTLY CANNOT SAVE TF OPTIMIZERS (tf.train)**
When using those ^, you need to recompile the model after loading and will lose state of optimizer.

##### Build a freshy:

In [25]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 219us/sample - loss: 1.1464 - accuracy: 0.6800
Epoch 2/5
1000/1000 [==============================] - 0s 109us/sample - loss: 0.4015 - accuracy: 0.8940
Epoch 3/5
1000/1000 [==============================] - 0s 112us/sample - loss: 0.2846 - accuracy: 0.9340
Epoch 4/5
1000/1000 [==============================] - 0s 144us/sample - loss: 0.1906 - accuracy: 0.9580
Epoch 5/5
1000/1000 [==============================] - 0s 118us/sample - loss: 0.1486 - accuracy: 0.9700


In [26]:
import time
saved_model_path = './saved_models/{}'.format(int(time.time()))

tf.keras.experimental.export_saved_model(model, saved_model_path)
saved_model_path

W0418 12:34:49.216258   624 deprecation.py:323] From C:\Users\Jessy\Anaconda3\lib\site-packages\tensorflow\python\saved_model\signature_def_utils_impl.py:253: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
W0418 12:34:49.217267   624 tf_logging.py:161] Export includes no default signature!
W0418 12:34:49.800121   624 tf_logging.py:161] Export includes no default signature!


'./saved_models/1555612487'

##### List of saved models:

In [28]:
ls saved_models

 Volume in drive C is OS
 Volume Serial Number is 9C33-2CCC

 Directory of C:\Users\Jessy\dev\machine-learning\TF 2.0 save and restore\saved_models

04/18/2019  12:34 PM    <DIR>          .
04/18/2019  12:34 PM    <DIR>          ..
04/18/2019  12:34 PM    <DIR>          1555612487
               0 File(s)              0 bytes
               3 Dir(s)  803,269,509,120 bytes free


##### Reload a fresh keras model from saved model:

In [29]:
new_model = tf.keras.experimental.load_from_saved_model(saved_model_path)
new_model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.predict(test_images).shape

(1000, 10)

In [32]:
#have to compile model before evaluation
#not required if it's only being deployed
new_model.compile(optimizer=model.optimizer, #keep loaded optimizer
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

loss, acc = new_model.evaluate(test_images, test_labels)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 89us/sample - loss: 0.4085 - accuracy: 0.8670
Restored model, accuracy: 86.70%


##### Fin